# 9. scikit-learn 入門

一般的に、機械学習は以下の手順で行う

1. データセットの準備
2. モデルを決める
3. 目的関数を決める
4. 最適化手法を選択する
5. モデルを訓練する

### 1.1 データを準備する

In [31]:
# dataset
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.pipeline import Pipeline

dataset = load_boston()

### 1.2 データを「入力値」と「目標値」に分割する

In [2]:
# divide data, target
x = dataset.data
t = dataset.target

### 1.3 入力値と目標値の次元数を確認

In [3]:
# confirm data
x.shape

(506, 13)

In [4]:
t.shape

(506,)

### 1.4 「訓練データ」と「テストデータ」に分割する

In [5]:
# split data train, test
x_train, x_test, t_train, t_test = \
    train_test_split(x, t, test_size=.3, random_state=0)

### 2.1 「モデル」を決める

In [6]:
# create liner model
clc = LinearRegression()

- 上記コードで「2.モデルの決定」「3.目的関数の決定」「4.最適化手法の選択」を全て行っている
- `LinearRegression`は線形回帰を行うモデルです。

### 5.1 モデルの訓練

In [7]:
# train model
clc.fit(x_train, t_train)

LinearRegression()

### 5.2 重み(w)とバイアス(b)の確認

In [8]:
# check weight
clc.coef_

array([-1.21310401e-01,  4.44664254e-02,  1.13416945e-02,  2.51124642e+00,
       -1.62312529e+01,  3.85906801e+00, -9.98516565e-03, -1.50026956e+00,
        2.42143466e-01, -1.10716124e-02, -1.01775264e+00,  6.81446545e-03,
       -4.86738066e-01])

In [9]:
# check bias
clc.intercept_

37.93710774183309

### 5.3 決定係数の確認

In [10]:
# check R2 score
clc.score(x_train, t_train)

0.7645451026942549

- 決定係数は1に近いほどモデルがデータに当てはまっていることを示す

### 6.1 新しい入力値から予測する

In [11]:
# 1行目のテスト入力値から予測
clc.predict(x_test[:1])

array([24.9357079])

In [12]:
# 1行目のテスト目標値
t_test[0]

22.6

### 6.2 モデルの評価

In [13]:
# check R2 score of test data
clc.score(x_test, t_test)

0.6733825506400171

### 7.1 データの前処理

- 前処理は、欠損値の補完、外れ値の除去、特徴量選択、正規化などの手法がある
- ここでは標準化を行う

In [14]:
# do standard
scaler = StandardScaler()

scaler.fit(x_train)

StandardScaler()

### 7.2 平均と分散の確認

In [15]:
# check mean and variance
scaler.mean_

array([3.35828432e+00, 1.18093220e+01, 1.10787571e+01, 6.49717514e-02,
       5.56098305e-01, 6.30842655e+00, 6.89940678e+01, 3.76245876e+00,
       9.35310734e+00, 4.01782486e+02, 1.84734463e+01, 3.60601186e+02,
       1.24406497e+01])

In [16]:
scaler.var_

array([6.95792305e+01, 5.57886665e+02, 4.87753572e+01, 6.07504229e-02,
       1.33257561e-02, 4.91423928e-01, 7.83932705e+02, 4.26314655e+00,
       7.49911344e+01, 2.90195600e+04, 4.93579208e+00, 7.31040807e+03,
       4.99634123e+01])

### 7.3 データ全てを標準化

In [17]:
# do scaler
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### 7.4 標準化したデータでモデルの訓練

In [18]:
# train model
clc.fit(x_train_scaled, t_train)

LinearRegression()

In [19]:
# check score of train data
clc.score(x_train_scaled, t_train)

0.7645451026942549

In [20]:
# test data
clc.score(x_test_scaled, t_test)

0.6733825506400195

### 8.1 べき変換

- 標準化では特に改善しなかったので、[べき変換](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html?highlight=powertransformer#sklearn.preprocessing.PowerTransformer)を行う

In [27]:
# learn poser transform from train data
scaler = PowerTransformer()
scaler.fit(x_train)

# do power transform train and test data
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

# fit model
clc.fit(x_train_scaled, t_train)

LinearRegression()

### 8.2 決定係数の導出

In [28]:
# R2 score of train data
clc.score(x_train_scaled, t_train)

0.7859862562650238

In [29]:
# test data
clc.score(x_test_scaled, t_test)

0.7002856552456189

- テストデータの決定係数が改善された

### 9.1 パイプライン

- scikit-learnは`fit()`メソッド`でインターフェースが統一されている
- パイプラインによって、上記の`fit()`メソッドなど一連の処理を結合できる

In [32]:
# create pipeline
pipeline = Pipeline([
    ('scaler', PowerTransformer()),
    ('clc', LinearRegression()),
])

### 9.2 べき変換 => モデルの学習

In [34]:
# do powertransform, then regression
pipeline.fit(x_train, t_train)

Pipeline(steps=[('scaler', PowerTransformer()), ('clc', LinearRegression())])

### 9.3 訓練データとテストデータのスコア

In [35]:
# train
pipeline.score(x_train, t_train)

0.7859862562650238

In [36]:
# test
pipeline.score(x_test, t_test)

0.7002856552456189